<a href="https://colab.research.google.com/github/data-IA-2022/FAO_Project_Tarik_FengFeng/blob/main/Copie_de_FAO_project_Fengfeng_Tarik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Data Analyse au sein de la FAO (Food and Agriculture Organization)</h1>

<h2>Contexte du projet</h2>

Au sein du service Data Analytics and Prospective de la FAO, vous êtes en charge d'analyser les données produites par les Nations Unies.

Ces analyses conduisent à des rapports de synthèse à destination des décideurs.

Pour êtes également en charge de la partie Prospective, c'est à dire donner des tendances et des évolutions probables quant à la production agricole, à l'alimentation et la nutrition mondiales.

In [11]:
# Copie des fichiers csv du repertoire github et suppréssions des ancien en cas de mise à jour

!rm -r /content/FAO_Project_Tarik_FengFeng
!git clone https://github.com/data-IA-2022/FAO_Project_Tarik_FengFeng.git

Cloning into 'FAO_Project_Tarik_FengFeng'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 125 (delta 14), reused 0 (delta 0), pack-reused 95
Receiving objects: 100% (125/125), 1.18 MiB | 4.40 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [12]:
#importation des librairies python
import os
import pandas as pd
import glob

In [13]:
# Récupération des chemin vers les fichiers csv
csv_path = '/content/FAO_Project_Tarik_FengFeng/fao_2013'
def find_csv_filenames(path_to_dir=os.getcwd(), extension="csv"):
    filenames = []
    for name in glob.glob(f'{path_to_dir}/*.{extension}'):
      filenames.append(name)
    return filenames
all_csv_files= find_csv_filenames(path_to_dir=csv_path)
print(all_csv_files)

['/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_pays-continents.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_cereal.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_population.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_vegetal.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_animal.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_sous_alimentation.csv']


In [33]:
# Création d'un dictionaire avec nom des fichiers csv comme Key et dataframe comme value

col_names= ['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole']
df_dict = {}
for f in all_csv_files:
  df_name= f.split("/")[-1].replace(".csv", "").split("_")[-1]
  if "FAOSTAT_2013" in f:
    df_dict[f'{df_name}'] = pd.read_csv(f, encoding='utf-8', quotechar='"', delimiter=',', names= col_names, header=0)
  else:
    df_dict[f'{df_name}'] = pd.read_csv(f, encoding='utf-8', quotechar='"', delimiter=',')

print(df_dict.keys())

dict_keys(['pays-continents', 'cereal', 'population', 'vegetal', 'animal', 'alimentation'])


In [34]:
# Définition des Dataframes

animal= df_dict["animal"]
cereal= df_dict["cereal"]
population= df_dict["population"]
alimentation= df_dict["alimentation"]
vegetal= df_dict["vegetal"]
pays_continents= df_dict["pays-continents"]

In [35]:
# Définir les codes pays comme index des Df
animal.index= animal["Code Pays"]
animal.index.name= "Code_Pays_id"
cereal.index= cereal["Code Pays"]
cereal.index.name= "Code_Pays_id"
population.index= population["Code Pays"]
population.index.name= "Code_Pays_id"
alimentation.index= alimentation["Code Pays"]
alimentation.index.name= "Code_Pays_id"
vegetal.index= vegetal["Code Pays"]
vegetal.index.name= "Code_Pays_id"
pays_continents.index= pays_continents["Code Pays"]
pays_continents.index.name= "Code_Pays_id"

# Nouvelle section

In [36]:
# nb lignes et nb colonnes de nos df
print(animal.shape)
print(cereal.shape)
print(population.shape)
print(alimentation.shape)
print(vegetal.shape)
print(pays_continents.shape)

(37166, 14)
(16057, 14)
(171, 14)
(204, 14)
(104871, 14)
(1941, 7)


In [52]:
# Join population et cereal production et pays continents
df_pop_cerealprod = population.join(cereal, on= "Code Pays",how="left", lsuffix="_pop", rsuffix="_cer" )
df_pop_cerealprodblé = df_pop_cerealprod.query("Produit_cer in ('Blé') and Élément_cer in ('Production')")
pays_continents= pays_continents.query("`Code Groupe de pays` in (5200,5100,5300,5500,5400) ")
#print(pays_continents)
df_pop_cerealprodblé = df_pop_cerealprodblé.join(pays_continents, on= "Code Pays" ,how="left", lsuffix="_popcer", rsuffix="_conti" )
df_pop_cerealprodblé = df_pop_cerealprodblé[["Valeur_pop","Valeur_cer","Pays_pop","Groupe de pays"]]
print(df_pop_cerealprodblé)

              Valeur_pop  Valeur_cer                            Pays_pop  \
Code_Pays_id                                                               
2                  30552      5169.0                         Afghanistan   
3                   3173       294.0                             Albania   
4                  39208      3299.0                             Algeria   
7                  21472         4.0                              Angola   
9                  41446      9315.0                           Argentina   
...                  ...         ...                                 ...   
235                28934      6842.0                          Uzbekistan   
236                30405         0.0  Venezuela (Bolivarian Republic of)   
249                24407       232.0                               Yemen   
251                14539       274.0                              Zambia   
181                14150        25.0                            Zimbabwe   

           

In [54]:
# Graphiques scatter plot de la production de blé par rapport à la population par pays

# Error divide by zero encountered in log prendre les valeurs supérieur


# Avec plotly
import plotly.express as px
import plotly.io as io
import numpy as np


io.renderers.default='colab'
df = df_pop_cerealprodblé
fig = px.scatter(df, x="Valeur_pop", y="Valeur_cer",title="Production de Blé par Population par Pays", hover_name="Pays_pop" , color="Groupe de pays", log_x=True, log_y=True)
fig.show()

In [58]:
# Creation data frame pour pareto population
# Join population et  pays continents
df_pareto_pop =  population.join(pays_continents, on= "Code Pays",how="left", lsuffix="_pop", rsuffix="_conti" )


df_pareto_pop  = df_pareto_pop [["Valeur","Pays_conti","Groupe de pays"]]
print(df_pareto_pop)
#print(df_pop_cerealprod)
#df_pop_cerealprod["sum_val_cer"]=df_pop_cerealprod.groupby(by="Groupe de pays")["Valeur_cer"].sum()

#print(df_pop_cerealprod.groupby(by="Groupe de pays")["Valeur_cer"].sum())

              Valeur                              Pays_conti Groupe de pays
Code_Pays_id                                                               
2              30552                             Afghanistan           Asie
3               3173                                 Albanie         Europe
4              39208                                 Algérie        Afrique
7              21472                                  Angola        Afrique
8                 90                      Antigua-et-Barbuda      Amériques
...              ...                                     ...            ...
236            30405  Venezuela (République bolivarienne du)      Amériques
237            91680                                Viet Nam           Asie
249            24407                                   Yémen           Asie
251            14539                                  Zambie        Afrique
181            14150                                Zimbabwe        Afrique

[171 rows x